# Set Bert

In [26]:
from transformers import AutoTokenizer, DistilBertModel
tokenizer = AutoTokenizer.from_pretrained("distilbert/distilbert-base-multilingual-cased")
distilBertModel = DistilBertModel.from_pretrained("distilbert/distilbert-base-multilingual-cased")

In [2]:
# get some vocab info
import random
random_tokens = random.sample(list(tokenizer.vocab), 10)
random_ids = [tokenizer.vocab[token] for token in random_tokens]

print("{0:20}{1:15}".format("token", "id"))
print("-" * 25)
for t, i in zip(random_tokens, random_ids):
    print("{0:15}{1:10}".format(t, i))

token               id             
-------------------------
asya                23795
##ingan             30408
ליצור               75899
Tidligere          101531
Júlio              104835
этим                37266
khi                 12072
estiu               59920
kullanılır          70408
Трудового           89395


In [27]:
distilBertModel.eval()

DistilBertModel(
  (embeddings): Embeddings(
    (word_embeddings): Embedding(119547, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (transformer): Transformer(
    (layer): ModuleList(
      (0-5): 6 x TransformerBlock(
        (attention): MultiHeadSelfAttention(
          (dropout): Dropout(p=0.1, inplace=False)
          (q_lin): Linear(in_features=768, out_features=768, bias=True)
          (k_lin): Linear(in_features=768, out_features=768, bias=True)
          (v_lin): Linear(in_features=768, out_features=768, bias=True)
          (out_lin): Linear(in_features=768, out_features=768, bias=True)
        )
        (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
        (ffn): FFN(
          (dropout): Dropout(p=0.1, inplace=False)
          (lin1): Linear(in_features=768, out_features=3072, bias=True)
          (lin2): L

In [13]:
input = tokenizer(["嗨", "hello"], return_tensors="pt", padding=True, truncation=True)    # Not support batch str input (because of torch.Tensor)
seqTensor = distilBertModel(**input, output_hidden_states=True)

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


In [5]:
print(seqTensor.hidden_states[0].shape)

torch.Size([1, 3, 768])


In [6]:
print(len(seqTensor.hidden_states))

7


In [14]:
print(input)

{'input_ids': tensor([[  101,   100,   102,     0],
        [  101, 61694, 10133,   102]]), 'attention_mask': tensor([[1, 1, 1, 0],
        [1, 1, 1, 1]])}


# Set BertDecoder

In [28]:
from utils.DevConf import DevConf
devConf = DevConf('cpu')

In [29]:
from utils.AttnBlocksConf import AttnBlocksConf

In [30]:
from model.BertDecoder.SentiClassifier import SentiClassifier
from utils.const import BlockType

In [31]:
mapper = SentiClassifier(AttnBlocksConf(768, 12, 6), BlockType.LAST, devConf=devConf)

In [17]:
ans = mapper.forward(seqTensor)
# print(ans.shape)
print(ans)

tensor([[-0.9691, -0.9359,  0.6656, -0.0389,  0.0653, -0.1121,  0.5723, -0.7163,
         -0.0133,  1.3693, -0.2713,  0.3737, -0.6853, -0.6463, -2.1772,  0.6569,
          0.2081, -0.0243, -0.5727, -2.3854,  0.5580,  2.1182, -0.7764,  0.2830,
         -0.5762,  1.2624,  1.4614, -0.4876, -0.3879,  0.8895,  0.3116, -0.9578,
         -1.5795, -0.5073,  0.8578, -0.1698, -0.3321, -1.2820, -2.0235, -0.4376,
          0.6965,  0.3917, -1.3476,  1.0086,  0.8861, -1.1901, -0.5668, -0.5730,
         -0.2863, -0.2144,  0.0369, -0.3639, -0.8184, -0.1639, -0.3035, -1.2355,
          0.4606, -2.2544, -0.4324,  0.2711, -1.7094, -1.1850, -0.0925,  0.0696,
         -1.4869,  0.2667, -0.6693, -1.5943, -1.1482, -0.2062, -1.4222,  0.6467,
          0.1574, -0.0290,  2.0798, -0.4745,  0.2181, -0.1663, -1.4687, -0.9155,
         -0.4516, -1.0438, -0.8063, -1.7207,  0.7113, -0.2269,  1.1177,  0.8830,
          0.5836,  1.4282,  0.0169,  1.6410, -0.9409, -1.4751,  0.6818, -0.1149,
         -2.1822, -1.4398,  

In [18]:
print(ans.shape)

torch.Size([1, 768])


# Combination Model

In [32]:
from model.CombinationModel import CombinationModel

In [33]:
from torch import nn

In [34]:
cModel = CombinationModel(
    # tokenizer=tokenizer,
    distilBert=distilBertModel,
    decoder=mapper,
    outputProject=nn.Linear(768, 5, device=devConf.device, dtype=devConf.dtype)
)

In [37]:
cModel.forward(**tokenizer("嗨", return_tensors="pt"), returnAttnWeight=True)

(tensor([[0.0764, 0.3002, 0.1742, 0.1757, 0.2736]], grad_fn=<SoftmaxBackward0>),
 tensor([[[0.3497, 0.3305, 0.3199]]], grad_fn=<MeanBackward1>))

# Prepare Data

In [2]:
from torchtext.datasets import IMDB
from torchtext.data.functional import to_map_style_dataset

In [23]:
trainData = iter(IMDB(root="./data", split='train'))

In [4]:
print(next(trainData)[1])

I rented I AM CURIOUS-YELLOW from my video store because of all the controversy that surrounded it when it was first released in 1967. I also heard that at first it was seized by U.S. customs if it ever tried to enter this country, therefore being a fan of films considered "controversial" I really had to see this for myself.<br /><br />The plot is centered around a young Swedish drama student named Lena who wants to learn everything she can about life. In particular she wants to focus her attentions to making some sort of documentary on what the average Swede thought about certain political issues such as the Vietnam War and race issues in the United States. In between asking politicians and ordinary denizens of Stockholm about their opinions on politics, she has sex with her drama teacher, classmates, and married men.<br /><br />What kills me about I AM CURIOUS-YELLOW is that 40 years ago, this was considered pornographic. Really, the sex and nudity scenes are few and far between, eve

In [12]:
from torch import Tensor
import torch

In [22]:
def collate_fn(batch)->tuple[Tensor, Tensor, Tensor]:
    target =[]
    datas = []

    for (label, data) in batch:
        datas.append(data)
        target.append(0 if label == 'neg' else 1)

    encoding = tokenizer(datas, return_tensors='pt', padding=True, truncation=True, max_length=512)
    ids = encoding['input_ids']
    mask = encoding['attention_mask']
    
    return ids, mask, torch.tensor(target, dtype=torch.int16)

In [7]:
from torch.utils.data import DataLoader

In [24]:
trainLoader = DataLoader(to_map_style_dataset(trainData), collate_fn=collate_fn, batch_size=16, shuffle=True)

In [25]:
next(iter(trainLoader))

(tensor([[  101, 10747, 18379,  ...,     0,     0,     0],
         [  101, 17443, 49307,  ..., 10833, 10124,   102],
         [  101, 76783, 10230,  ...,     0,     0,     0],
         ...,
         [  101,   146, 10134,  ...,     0,     0,     0],
         [  101, 14027, 18379,  ...,     0,     0,     0],
         [  101, 12689,   169,  ...,   118, 18923,   102]]),
 tensor([[1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 1, 1, 1],
         [1, 1, 1,  ..., 0, 0, 0],
         ...,
         [1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 1, 1, 1]]),
 tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], dtype=torch.int16))

## deprecated

In [ ]:
from torchtext.data.utils import get_tokenizer

In [3]:
tokenizer = get_tokenizer('basic_english')

In [4]:
print(tokenizer(next(trainData)[1]))

['i', 'rented', 'i', 'am', 'curious-yellow', 'from', 'my', 'video', 'store', 'because', 'of', 'all', 'the', 'controversy', 'that', 'surrounded', 'it', 'when', 'it', 'was', 'first', 'released', 'in', '1967', '.', 'i', 'also', 'heard', 'that', 'at', 'first', 'it', 'was', 'seized', 'by', 'u', '.', 's', '.', 'customs', 'if', 'it', 'ever', 'tried', 'to', 'enter', 'this', 'country', ',', 'therefore', 'being', 'a', 'fan', 'of', 'films', 'considered', 'controversial', 'i', 'really', 'had', 'to', 'see', 'this', 'for', 'myself', '.', 'the', 'plot', 'is', 'centered', 'around', 'a', 'young', 'swedish', 'drama', 'student', 'named', 'lena', 'who', 'wants', 'to', 'learn', 'everything', 'she', 'can', 'about', 'life', '.', 'in', 'particular', 'she', 'wants', 'to', 'focus', 'her', 'attentions', 'to', 'making', 'some', 'sort', 'of', 'documentary', 'on', 'what', 'the', 'average', 'swede', 'thought', 'about', 'certain', 'political', 'issues', 'such', 'as', 'the', 'vietnam', 'war', 'and', 'race', 'issues', 

# Train

In [ ]:
import torch

In [ ]:
loss_fn = torch.nn.CrossEntropyLoss()

In [ ]:
model = SentiClassifier(distilBertModel=distilBertModel, tokenizer=tokenizer, output_dim=2)

In [ ]:
print(model("hello world!"))

tensor([[0.4752, 0.5248]], grad_fn=<SoftmaxBackward0>)


In [ ]:
import torch
from einops import repeat

In [ ]:
x = torch.ones(2, requires_grad=True)
y = repeat(x, "l -> b l", b=2) * 2
# with torch.no_grad():
#     for i in range(3):
#         y = 2*y
#     # x.retain_grad()
y1 = 2*y[0]
y2 = y[1]
z = y1.sum() + y2.sum()

In [ ]:
print(x)
print(y)

tensor([1., 1.], requires_grad=True)
tensor([[4., 4.],
        [4., 4.]], grad_fn=<MulBackward0>)


In [ ]:
z.backward()

In [ ]:
print(x.grad)

tensor([6., 6.])


In [2]:
strlist = ["a", "b", "c", "d", "e"]
concatStr = ""
for i in strlist:
    concatStr+=i
    concatStr+=" "
print(concatStr)

a b c d e 


## Loss function

In [1]:
from torch import nn

In [30]:
loss_fn = nn.BCELoss()

In [4]:
import torch

In [31]:
loss_fn(torch.tensor([[0.1, 0.9, 0.2], [0.1, 0.9, 0.5]]), torch.tensor([[1, 2], [1]])) 

ValueError: expected sequence of length 2 at dim 1 (got 1)